<a href="https://colab.research.google.com/github/VellummyilumVinoth/Train_And_Test_Using_ALBERT/blob/main/FinetunedAlbert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# from transformers import AutoTokenizer, AutoModelForMaskedLM

# # Load tokenizer for CodeBERTa small v1
# tokenizer = AutoTokenizer.from_pretrained("huggingface/codeberta-small-v1")

# # Load ALBERT base v2 model
# model = AutoModelForMaskedLM.from_pretrained("albert-base-v2")


In [2]:
# # Tokenize input sentence
# input_sentence = "This is a sample sentence to tokenize."
# input_ids = tokenizer.encode(input_sentence, add_special_tokens=True, return_tensors="pt")

# # Pass the tokenized input to the ALBERT model
# outputs = model(input_ids=input_ids)

# # Get the predicted token probabilities
# predicted_probabilities = outputs.logits.softmax(dim=-1)[0, :-1]


In [3]:
# from transformers import AutoTokenizer

# # Load the Hugging Face/CodeBERTa Small v1 tokenizer
# codeberta_tokenizer = AutoTokenizer.from_pretrained("huggingface/codeberta-small-v1")

# # Load the ALBERT Base v2 tokenizer
# albert_tokenizer = AutoTokenizer.from_pretrained("albert-base-v2")

# # Get the CodeBERTa vocabulary
# codeberta_vocab = codeberta_tokenizer.get_vocab()

# # Create a new vocabulary dictionary for the ALBERT tokenizer
# albert_vocab = {}

# # Convert each token in the CodeBERTa vocabulary to its corresponding ID in the ALBERT vocabulary
# for token in codeberta_vocab:
#     albert_vocab[token] = albert_tokenizer.convert_tokens_to_ids(codeberta_tokenizer.tokenize(token))[0]

# # Create a new instance of the ALBERT tokenizer with the new vocabulary
# new_albert_tokenizer = AutoTokenizer.from_pretrained("albert-base-v2", vocab=albert_vocab)

# # Save the new tokenizer
# new_albert_tokenizer.save_pretrained("albert-with-codeberta-vocab")


In [4]:
# text = "int[] intList = getNumbers();"

# tokenizer = AlbertTokenizer.from_pretrained("/content/albert-with-codeberta-tokenization")


# tokens1 = tokenizer.tokenize(text)

# # tokens = [token.replace('Ġ', '') for token in tokens]

# print(tokens1)

In [5]:
# from transformers import AlbertTokenizer

# # Load the pre-trained tokenizer
# tokenizer = AutoTokenizer.from_pretrained('huggingface/codeberta-small-v1')

# # Get the vocabulary dictionary
# vocab_dict = tokenizer.get_vocab()

# # Save the vocabulary dictionary to a file
# with open('vocab.txt', 'w') as f:
#     for token, idx in vocab_dict.items():
#         f.write(f"{token} {idx}\n")


In [6]:
# from transformers import AutoTokenizer, AlbertTokenizer
# albert_tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
# codeberta_tokenizer = AutoTokenizer.from_pretrained('huggingface/codeberta-small-v1')
# albert_vocab = albert_tokenizer.get_vocab()
# albert_special_tokens = albert_tokenizer.all_special_tokens
# new_vocab = {}
# for token in codeberta_tokenizer.get_vocab():
#     if token in albert_vocab:
#         new_vocab[token] = albert_vocab[token]
# new_special_tokens = list(set(albert_special_tokens + codeberta_tokenizer.all_special_tokens))
# new_tokenizer = AlbertTokenizer(vocab_file='/content/vocab.txt')
# new_tokenizer.add_tokens(list(new_vocab.keys()))
# new_tokenizer.add_tokens(new_special_tokens)


In [7]:
# text = "string tokenUrl = config.tokenUrl;"
# encoded = new_tokenizer.encode_plus(text, max_length=128, padding='max_length', truncation=True)
# encoded
# inputs = new_tokenizer.tokenize(text)
# print(inputs)

In [8]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [9]:
import locale
print(locale.getpreferredencoding())

UTF-8


In [10]:
import locale
def getpreferredencoding(do_setlocale = True):
    return "UTF-8"
locale.getpreferredencoding = getpreferredencoding

In [11]:

!pip install transformers

!pip install sentencepiece


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 56.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 104.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 27.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.6 MB/s eta 0:00:00


In [12]:
from transformers import AlbertForMaskedLM, AlbertTokenizer,AutoTokenizer,BertTokenizerFast,AlbertTokenizerFast,RobertaTokenizer,DistilBertTokenizer,RobertaTokenizerFast

# Load the ALBERT tokenizer and model
# tokenizer = RobertaTokenizerFast.from_pretrained('roberta-base')
tokenizer1 = RobertaTokenizer.from_pretrained("microsoft/codebert-base-mlm")
tokenizer2 = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
# tokenizer = AutoTokenizer.from_pretrained('microsoft/deberta-v2-xlarge')
tokenizer3 = RobertaTokenizer.from_pretrained('roberta-large')

# tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
tokenizer4 = BertTokenizerFast.from_pretrained('bert-base-uncased')
tokenizer = RobertaTokenizerFast.from_pretrained('huggingface/CodeBERTa-small-v1')
tokenizer5 = AutoTokenizer.from_pretrained('neulab/codebert-java')
tokenizer7 = AlbertTokenizer.from_pretrained("albert-base-v2")
# tokenizer.pad_token = tokenizer.eos_token

model = AlbertForMaskedLM.from_pretrained('albert-base-v2')

In [13]:
text = "Client dbClient1 = check new (jdbcUrl, user, password, options);"
# text = text.lower()

tokens = tokenizer.tokenize(text)
tokens1 = tokenizer1.tokenize(text)
tokens2 = tokenizer2.tokenize(text)
tokens3 = tokenizer3.tokenize(text)
tokens4 = tokenizer4.tokenize(text)
tokens5 = tokenizer5.tokenize(text)
tokens7 = tokenizer7.tokenize(text)

tokens = [token.replace('Ġ', '').lower() for token in tokens]
tokens1 = [token.replace('Ġ', '').lower() for token in tokens1]
tokens2 = [token.replace('Ġ', '').lower() for token in tokens2]
tokens3 = [token.replace('Ġ', '').lower() for token in tokens3]
tokens4 = [token.replace('Ġ', '').lower() for token in tokens4]
tokens5 = [token.replace('Ġ', '').lower() for token in tokens5]
tokens7 = [token.replace('▁', '').lower() for token in tokens7]


print("huggingface",tokens)
print("microsoft",tokens1)
print("distilbert",tokens2)
print("codebert",tokens4)
print("neula",tokens5)
print("albert",tokens7)



huggingface ['client', 'db', 'client', '1', '=', 'check', 'new', '(', 'jdbc', 'url', ',', 'user', ',', 'password', ',', 'options', ');']
microsoft ['client', 'db', 'client', '1', '=', 'check', 'new', '(', 'j', 'db', 'c', 'url', ',', 'user', ',', 'password', ',', 'options', ');']
distilbert ['client', 'db', '##cl', '##ient', '##1', '=', 'check', 'new', '(', 'jd', '##bc', '##ur', '##l', ',', 'user', ',', 'password', ',', 'options', ')', ';']
codebert ['client', 'db', '##cl', '##ient', '##1', '=', 'check', 'new', '(', 'jd', '##bc', '##ur', '##l', ',', 'user', ',', 'password', ',', 'options', ')', ';']
neula ['client', 'db', 'client', '1', '=', 'check', 'new', '(', 'j', 'db', 'c', 'url', ',', 'user', ',', 'password', ',', 'options', ');']
albert ['client', '', 'd', 'bc', 'lien', 't', '1', '=', 'check', 'new', '', '(', 'j', 'db', 'cur', 'l', ',', 'user', ',', 'password', ',', 'options', ')', ';']


In [14]:
import csv

variable_names = []
statements = []

with open('output.csv', 'r') as file:
    reader = csv.reader(file)
    for row in reader:
        # Check if the row contains at least two columns
        if len(row) >= 2:
            # Append the variable name and source statement to their respective lists
            variable_names.append(row[0].lower())
            statements.append(row[1])


In [15]:
variable_names

['intarray',
 'persons',
 'personlist',
 'personarray',
 'intarray',
 'numbers',
 'numbersarray',
 'numberslist',
 'count',
 'name',
 'age',
 'isauthorized',
 'israining',
 'isflag',
 'salary',
 'names',
 'payload',
 'datelist',
 'datearray',
 'totalprice',
 'message',
 'isfound',
 'employees',
 'students',
 'order',
 'student',
 'person',
 'persons',
 'persons',
 'num',
 'headers',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'textprompt',
 'completionres',
 'imageprmt',
 'imageres',
 'product',
 'prodobj',
 'pid',
 'lastrowindex',
 'row',
 'message',
 'payload',
 'message',
 'payload',
 'message',
 'payload',
 'outputstream',
 'streamvalue',
 'nextrecord',
 'clientendpoint',
 'returnar

In [16]:
statements

['int[] intArray = getNumbers();',
 'Person[] persons = getPersons();',
 'Person[] personList = getPersons();',
 'int[] personArray = getPersons();',
 'int[] intList = getNumbers();',
 'int[] numbers = getNumbers();',
 'int[] numbersArray = getNumbers();',
 'int[] numbersList = getNumbers();',
 'int count = getCount();',
 'string name = getName();',
 'int age = getAge();',
 'boolean isAuthorized = checkAuthorization();',
 'boolean isRaining = false;',
 'boolean isFlag = true;',
 'float salary = getSalary();',
 'string[] names = getNames();',
 'json payload = getPayload();',
 'Date[] dateList = [];',
 'Date[] dateArray = [];',
 'float totalPrice = calculateTotalPrice();',
 'string message = getMessage();',
 'boolean isFound = findItem();',
 'Employee[] employees = getEmployees();',
 'Students[] students = getStudents();',
 'Order order = getOrder();',
 'Student student;',
 "Person person = {name: 'John'};",
 'Person[] persons = getPersons();',
 "Person[] persons = getPersons(10, 'Sales'

In [17]:
import re

def clean_text(line):
    line = re.sub(r'-+',' ',line)
    line = re.sub(r'[^a-zA-Z, ]+'," ",line)
    line = re.sub(r'[ ]+'," ",line)
    line += ""
    return line

source_statements = []
len_lst = []
for line in statements:
    if len(line.split(" ")) >=0:
        line = clean_text(line)
        source_statements.append(line)
        len_lst.append(len(line.split(" ")))


In [18]:
source_statements

['int intArray getNumbers ',
 'Person persons getPersons ',
 'Person personList getPersons ',
 'int personArray getPersons ',
 'int intList getNumbers ',
 'int numbers getNumbers ',
 'int numbersArray getNumbers ',
 'int numbersList getNumbers ',
 'int count getCount ',
 'string name getName ',
 'int age getAge ',
 'boolean isAuthorized checkAuthorization ',
 'boolean isRaining false ',
 'boolean isFlag true ',
 'float salary getSalary ',
 'string names getNames ',
 'json payload getPayload ',
 'Date dateList ',
 'Date dateArray ',
 'float totalPrice calculateTotalPrice ',
 'string message getMessage ',
 'boolean isFound findItem ',
 'Employee employees getEmployees ',
 'Students students getStudents ',
 'Order order getOrder ',
 'Student student ',
 'Person person name John ',
 'Person persons getPersons ',
 'Person persons getPersons , Sales ',
 'int num ',
 'map string string headers ',
 'AddItemRequest message ',
 'var payload check self grpcClient executeSimpleRPC hipstershop Cart

In [19]:
# from huggingface_hub import add_space_secret
import pandas as pd
import torch
import os
import re
from transformers import AdamW, Trainer, TrainingArguments, DataCollatorForLanguageModeling

import random

class VariableNamesDataset(torch.utils.data.Dataset):
    def __init__(self, variable_names, source_statements, tokenizer, mask_probability):
        self.variable_names = variable_names
        self.source_statements = source_statements
        self.tokenizer = tokenizer
        self.mask_probability = mask_probability

    def __len__(self):
        return len(self.source_statements)
    
    def __getitem__(self, idx):
        variable_name = str(self.variable_names[idx])
        source_statement = str(self.source_statements[idx])

        print("variable names: ",len(variable_name))
        print("source statements: ",len(source_statement))

        # Tokenize the variable name and source statement
        variable_name_tokens = self.tokenizer.tokenize(variable_name)
        source_statement_tokens = self.tokenizer.tokenize(source_statement)

        variable_name_tokens = [token.replace('Ġ', '').lower() for token in variable_name_tokens]
        source_statement_tokens = [token.replace('Ġ', '').lower() for token in source_statement_tokens]

        print("Variable name tokens:", variable_name_tokens)
        print("Source statement tokens:", source_statement_tokens)

        # Select a variable name tokens to mask
        variable_name_indices = [i for i, token in enumerate(source_statement_tokens) if token in variable_name_tokens]

        print("Variable name indices:", variable_name_indices)

        num_to_mask = int(len(variable_name_indices)* self.mask_probability)
        indices_to_mask = random.sample(variable_name_indices, num_to_mask)

        print("Num to mask:", num_to_mask)
        print("Indices to mask:", indices_to_mask)

        # Replace the selected variable name tokens with the [MASK] token
        for i in indices_to_mask:
            source_statement_tokens[i] = '<mask>'

        masked_source_statement = ' '.join(source_statement_tokens)
        print("Masked source statement:", masked_source_statement)

        # Tokenize the masked source statement
        input_ids = self.tokenizer.encode(
            masked_source_statement,
            add_special_tokens=False,
            truncation=True,
            padding='max_length',
            max_length=128
        )

        print("Input IDs:", input_ids)

        return torch.tensor(input_ids)
 

In [20]:
# Split the data into train and test sets
train_size = int(len(source_statements) * 0.9)
train_variable_names = variable_names[:train_size]
train_source_statements = source_statements[:train_size]
test_variable_names = variable_names[train_size:]
test_source_statements = source_statements[train_size:]

In [21]:
# Prepare the data for training and testing
train_dataset = VariableNamesDataset(train_variable_names, train_source_statements, tokenizer, mask_probability = 1)

test_dataset = VariableNamesDataset(test_variable_names, test_source_statements, tokenizer, mask_probability = 0)

In [22]:
train_dataset[13]

variable names:  6
source statements:  20
Variable name tokens: ['is', 'flag']
Source statement tokens: ['boolean', 'is', 'flag', 'true', '']
Variable name indices: [1, 2]
Num to mask: 2
Indices to mask: [1, 2]
Masked source statement: boolean <mask> <mask> true 
Input IDs: [3274, 4, 4, 691, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


tensor([3274,    4,    4,  691,  225,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])

In [23]:
test_dataset[25]

variable names:  5
source statements:  35
Variable name tokens: ['value']
Source statement tokens: ['int', 'error', 'value', 'int', 'fromstring', 'exp', '']
Variable name indices: [2]
Num to mask: 0
Indices to mask: []
Masked source statement: int error value int fromstring exp 
Input IDs: [437, 670, 602, 664, 772, 492, 2464, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


tensor([ 437,  670,  602,  664,  772,  492, 2464,  225,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1])

In [24]:
import torch
torch.cuda.empty_cache()

In [25]:
# Prepare the data collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15
)

model.resize_token_embeddings(len(tokenizer))

# Set up training arguments
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    evaluation_strategy = "steps",   # Evaluation and Save happens every X steps
    eval_steps = 100,                # Evaluation happens every X steps
    save_steps = 1000,               # Save checkpoint every X steps
    num_train_epochs = 10,            # Total number of training epochs
    learning_rate = 1e-5,            # Learning rate
    per_device_train_batch_size=32,  # Batch size per device during training
    per_device_eval_batch_size=64,   # Batch size for evaluation
    warmup_steps = 500,              # Number of warmup steps
    weight_decay = 0.01,             # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_steps=1000,              # Log every X steps
)

# Instantiate the Trainer class and train the model
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    data_collator=data_collator,
)

trainer.train()


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


variable names:  4
source statements:  41
Variable name tokens: ['expr']
Source statement tokens: ['literal', 'expr', 'expr', 'startpos', ',', 'endpos', ',', 'value', '']
Variable name indices: [1, 2]
Num to mask: 2
Indices to mask: [1, 2]
Masked source statement: literal <mask> <mask> startpos , endpos , value 
Input IDs: [8920, 4, 4, 1148, 839, 3161, 835, 839, 3161, 602, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
variable names:  7
source statements:  27
Variable name tokens: ['message']
Source statement tokens: ['struct', 'array', 'request', 'message', '']
Variable name indices: [3]
Num to mask: 1
Indices to mask: [3]
Masked source statement: struct array request <mask> 
Inpu

Step,Training Loss,Validation Loss
100,No log,10.526242
200,No log,9.328520
300,No log,8.781389
400,No log,8.375866
500,No log,8.278205
600,No log,7.560510
700,No log,7.436945


Streaming output truncated to the last 5000 lines.
Variable name indices: [1, 3]
Num to mask: 2
Indices to mask: [1, 3]
Masked source statement: string <mask> ref <mask> 
Input IDs: [492, 4, 1872, 4, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
variable names:  5
source statements:  24
Variable name tokens: ['value']
Source statement tokens: ['result', 'value', 'data', 'value', '']
Variable name indices: [1, 3]
Num to mask: 2
Indices to mask: [3, 1]
Masked source statement: result <mask> data <mask> 
Input IDs: [760, 4, 681, 4, 225, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

TrainOutput(global_step=700, training_loss=8.037193777901786, metrics={'train_runtime': 711.3515, 'train_samples_per_second': 31.138, 'train_steps_per_second': 0.984, 'total_flos': 124826280345600.0, 'train_loss': 8.037193777901786, 'epoch': 10.0})

In [26]:
# Save the trained model and tokenizer
output_dir = './finetuned_albert'
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

model.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)


('./finetuned_albert/tokenizer_config.json',
 './finetuned_albert/special_tokens_map.json',
 './finetuned_albert/vocab.json',
 './finetuned_albert/merges.txt',
 './finetuned_albert/added_tokens.json',
 './finetuned_albert/tokenizer.json')

In [34]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM, AutoTokenizer,RobertaTokenizer,RobertaTokenizerFast
from tabulate import tabulate

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
model = AlbertForMaskedLM.from_pretrained(output_dir)
tokenizer = RobertaTokenizerFast.from_pretrained(output_dir)

# Define a sample masked statement
masked_statement = "int <mask> = getAge();"

# Tokenize the masked statement
input_ids = tokenizer.encode(masked_statement, add_special_tokens=False, return_tensors='pt')

# Find the position of the masked token
masked_token_index = torch.where(input_ids == tokenizer.mask_token_id)[1][0].item()

# Generate predictions for the masked token using the fine-tuned model
with torch.no_grad():
    outputs = model(input_ids)
    predictions = outputs[0]

# Get the top 5 predictions and their probability scores from the fine-tuned model
probs_ft = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
top_k_ft = torch.topk(probs_ft, k=5)

# Create a table with the top predictions and their probabilities from both models
table = [["Fine-Tuned Model", f"{tokenizer.mask_token}"] + [tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower() for idx in top_k_ft.indices],
         ["Probability", ""] + [f"{probs_ft[idx].item():.4f}" for idx in top_k_ft.indices]]

# Print the table
print(tabulate(table, headers="firstrow", tablefmt="fancy_grid"))


╒════════════════════╤══════════╤═════════╤════════╤════════╤════════╤════════╕
│ Fine-Tuned Model   │ <mask>   │   check │    int │      s │   type │   self │
╞════════════════════╪══════════╪═════════╪════════╪════════╪════════╪════════╡
│ Probability        │          │  0.2408 │ 0.2155 │ 0.0588 │ 0.0354 │ 0.0315 │
╘════════════════════╧══════════╧═════════╧════════╧════════╧════════╧════════╛


In [37]:
import torch
from transformers import AlbertTokenizer, AlbertForMaskedLM,BertTokenizerFast,AutoTokenizer,RobertaTokenizer

# Load the trained model and tokenizer
output_dir = './finetuned_albert'
finetuned_model = AlbertForMaskedLM.from_pretrained(output_dir)
# finetuned_tokenizer = BertTokenizerFast.from_pretrained(output_dir)
finetuned_tokenizer = RobertaTokenizer.from_pretrained(output_dir)


# Load the ALBERT model and tokenizer
base_model = AlbertForMaskedLM.from_pretrained('albert-base-v2')
# base_tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
base_tokenizer = RobertaTokenizer.from_pretrained('huggingface/CodeBERTa-small-v1')

base_model.resize_token_embeddings(len(base_tokenizer))

# Define a list of sample masked statements
masked_statements = ["int <mask> = getCount();", "Student <mask>;", "Person <mask> = {name: 'John'};", "int[] <mask> = getNumbers();", "Person[] <mask> = getPersons();", "Person[] <mask> = getPersons(10, 'Sales');"]

# Loop through each masked statement and generate predictions for both models
for masked_statement in masked_statements:
    print(f"Masked statement: {masked_statement}\n")
    # Tokenize the masked statement for the fine-tuned model
    input_ids = finetuned_tokenizer.encode(masked_statement, add_special_tokens=False, return_tensors='pt')
    masked_token_index = torch.where(input_ids == finetuned_tokenizer.mask_token_id)[1][0].item()
    with torch.no_grad():
        outputs = finetuned_model(input_ids)
        predictions = outputs[0]
    probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k = torch.topk(probs, k=5)
    # Print the top 5 predictions and their probability scores for the fine-tuned model
    print("Fine-tuned model predictions:")
    print("{:<20} {:<20}".format('Prediction', 'Probability'))
    for i, idx in enumerate(top_k.indices):
        token = finetuned_tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower()
        prob = top_k.values[i].item()
        print("{:<20} {:.4f}".format(token, prob))
    print("\n")
    
    # Tokenize the masked statement for the base model
    input_ids = base_tokenizer.encode(masked_statement, add_special_tokens=False, return_tensors='pt')
    masked_token_index = torch.where(input_ids == base_tokenizer.mask_token_id)[1][0].item()
    with torch.no_grad():
        outputs = base_model(input_ids)
        predictions = outputs[0]
    probs = torch.nn.functional.softmax(predictions[0, masked_token_index], dim=-1)
    top_k = torch.topk(probs, k=5)
    # Print the top 5 predictions and their probability scores for the base model
    print("Base model predictions:")
    print("{:<20} {:<20}".format('Prediction', 'Probability'))
    for i, idx in enumerate(top_k.indices):
        token = base_tokenizer.convert_ids_to_tokens([idx])[0].replace('Ġ', '').lower()
        prob = top_k.values[i].item()
        print("{:<20} {:.4f}".format(token, prob))
    print("\n")


Masked statement: int <mask> = getCount();

Fine-tuned model predictions:
Prediction           Probability         
check                0.1196
s                    0.0615
int                  0.0416
<unk>                0.0408
code                 0.0366


Base model predictions:
Prediction           Probability         
§                    0.0873
describes            0.0494
åģ                   0.0482
//                   0.0299
kam                  0.0273


Masked statement: Student <mask>;

Fine-tuned model predictions:
Prediction           Probability         
autoload             0.0802
autoincrement        0.0494
§                    0.0491
=                    0.0409
(?:[                 0.0296


Base model predictions:
Prediction           Probability         
§                    0.0717
(?:[                 0.0550
=                    0.0515
leg                  0.0442
isassignablefrom     0.0394


Masked statement: Person <mask> = {name: 'John'};

Fine-tuned model predictio